In [ ]:
using LowLevelFEM
import LowLevelFEM as FEM

gmsh.initialize()

In [21]:
using Plots

In [ ]:
gmsh.open("wave.geo")

In [23]:
#gmsh.fltk.run()

In [ ]:
body = FEM.material("body")
problem1 = FEM.Problem([body], type="PlaneStress")
problem2 = FEM.Problem([body], type="PlaneHeatConduction")

In [ ]:
K1 = FEM.stiffnessMatrix(problem1)

In [ ]:
M1 = FEM.massMatrix(problem1, lumped=true)

In [ ]:
K2 = FEM.heatConductionMatrix(problem2)

In [ ]:
C2 = FEM.heatCapacityMatrix(problem2, lumped=false)

In [29]:
load = FEM.load("left", fx=0)
f1 = FEM.loadVector(problem1, [load]);

In [30]:
hf = FEM.heatFlux("left", qn=0)
q2 = FEM.heatFluxVector(problem2, [hf]);

In [31]:
right1 = FEM.displacementConstraint("right", ux=0);

In [32]:
u0 = FEM.initialDisplacement(problem1, "body", ux=0, uy=0)
v0 = FEM.initialVelocity(problem1, "body", vx=1000)
FEM.initialVelocity!(problem1, "right", v0, vx=0);

In [33]:
FEM.applyBoundaryConditions!(problem1, K1, M1, f1, [right1])

In [ ]:
Tₘᵢₙ = FEM.smallestPeriodTime(K1, M1)
Δt = Tₘᵢₙ \ π

In [35]:
u1, v1, t1 = FEM.CDM(K1, M1, f1, u0, v0, 2Δt, Δt);

In [36]:
T0 = FEM.initialTemperature(problem2, "body", T=273);

In [ ]:
T2, t2 = FEM.FDM(K2, C2, q2, T0, 2Δt, Δt)

In [ ]:
LH2 = FEM.latentHeatMatrix(problem2, u1[:, 2], v1[:, 2], T0)

In [ ]:
T0 = FEM.initialTemperature(problem, "right", T=100)

In [ ]:
λₘᵢₙ = FEM.smallestEigenValue(K, C)

In [ ]:
λₘₐₓ = FEM.largestEigenValue(K, C)

In [ ]:
ϑ = 0.4
Δt = 2 / ((1 - 2ϑ) * λₘₐₓ)
ϑ = 0.5

In [ ]:
xy = FEM.FDMaccuracyAnalysis(λₘᵢₙ, λₘₐₓ, Δt, ϑ=ϑ)
plot(xy)

In [ ]:
T, t = FEM.FDM(K, C, q0, T0, 1000 * Δt, Δt, ϑ=ϑ)

In [499]:
T1 = FEM.showDoFResults(problem, T, "T", t=t, visible=true);

In [ ]:
p1, xy = FEM.plotOnPath(problem, "path", T1, plot=true)
plot(xy)

In [ ]:
gmsh.fltk.run()

In [502]:
gmsh.finalize()